In [1]:
%load_ext autotime
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [2]:
# Set this to False to rebuild filelist.txt and meta.csv
use_cache = False

In [3]:
# This cell will read a list of files from Nick/filelist.txt if it exists, or build a list of files and save it to that location
# This is useful for quickly loading the list of files without having to search the entire directory structure
# You will need to delete the file and rerun this cell if files are created, renamed or moved

filename = prefix + "Nick/filelist.txt"
if use_cache and os.path.isfile(filename):
    filelist = pd.read_csv(filename, header=None).iloc[:,0]
else:
    def find_files(root):
        return pd.Series(glob(prefix + root + "**/**", recursive=True)).str.replace(prefix, "")
    filelist = pd.concat(thread_map(find_files, ["Gabrielle", "MaxarImagery", "Retrolens", "SpatialData/Mosaics"]))
    if platform.system() == "Windows":
        filelist = filelist.str.replace("\\", "/", regex=False)
    filelist.to_csv(filename, index=False, header=False)
filelist

  0%|          | 0/4 [00:00<?, ?it/s]

0                                                                                   MaxarImagery/
1                                                                   MaxarImagery/Northland_Update
2                                                       MaxarImagery/Northland_Update/NinetyMile3
3                             MaxarImagery/Northland_Update/NinetyMile3/NinetyMile3_22OCT2023.tif
4                         MaxarImagery/Northland_Update/NinetyMile3/NinetyMile3_22OCT2023.tif.ovr
                                                   ...                                           
55498    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L02_R00000062_C00000065.tfw
55499    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L03_R00000020_C00000022.tfw
55500    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000125_C00000131.tif
55501    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000127_C00000133.tif
55502               

In [4]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}\w*.shp$', filename)
    return bool(match)

shapefiles = filelist[filelist.apply(check_filename)]
df = shapefiles.to_frame(name="filename")
df

,filename
1172,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_31AUG2005.shp
1174,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_29JAN2011.shp
1184,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_25DEC2015.shp
1188,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_08NOV2019.shp
1191,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_03NOV2013.shp
...,...
112754,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
112759,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
113860,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
113861,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_13FEB2021.shp


In [5]:
image_extensions = (".jpg", ".jp2", ".tif", ".TIF", ".png")

In [6]:
images = filelist[filelist.str.contains("Stack", case=False) & filelist.str.endswith(image_extensions)]
images

1084      MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
1085      MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
1087        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
1090        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
1104        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
114093            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
114099            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
114102            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
114103            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
114106            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Length: 2650, dtype: object

In [7]:
Gabrielle_images = filelist[filelist.str.startswith("Gabrielle") & filelist.str.endswith(image_extensions)]

In [8]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_", " "]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    if filename.startswith("Gabrielle"):
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
        return match, score
    else:
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=images, processor=fuzz_preprocess)
        return match, score

df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))

/tmp/ipykernel_3574401/4132008065.py:24: TqdmWarning: Iterable length 2796 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))


  0%|          | 0/2796 [00:00<?, ?it/s]

Perfect matches: 1913
Imperfect matches: 883


In [9]:
df = df.sort_values("match_score")
df

,filename,matched_image,match_score
33867,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111
34689,Retrolens/Waikato/PortJackson/Shorelines/PortJackson_23AUG1971.shp,Retrolens/Northland/TaupiriBay/Stack/TaupiriBay_23AUG1971.tif,61.538462
38165,Retrolens/Waikato/Thames/Shorelines/Thames_25JAN1973.shp,Retrolens/Auckland/PakiriBeach_North/Stack/PakiriBeach_North_15JAN1977_mosaic.jp2,62.307692
33360,Retrolens/Waikato/Raglan/Shorelines/Raglan_24FEB2013.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_20FEB2016.tif,62.500000
33343,Retrolens/Waikato/Raglan/Shorelines/Raglan_22MAR1944.shp,MaxarImagery/HighFreq/Otago/Aramoana/Imagery/Stack/Aramoana_20MAR2014.tif,62.500000
...,...,...,...
48888,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.000000
48892,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_beach.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.000000
49079,Retrolens/Taranaki/UrenuiRiver_North_AOI/Shorelines/UrenuiRiver_North_5MAR1943.shp,Retrolens/Taranaki/UrenuiRiver_North_AOI/Stack/UrenuiRiver_North_5MAR1943_mosaic.jp2,100.000000
48849,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_8NOV1970_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_8NOV1970_mosaic.jp2,100.000000


In [10]:
def get_shapefile_meta(tup):
    i, row = tup
    filename = row.filename
    source = "Unknown"
    try:
        shapefile = gpd.read_file(prefix+filename)
    except:
        print(f"Can't read {filename}")
        return {"Source": source, "n_lines": 0}
    if "Source" not in shapefile.columns:
        if filename.startswith("Retrolens"):
            source = "RL"
        elif filename.startswith("MaxarImagery/HighFreq"):
            source = "MAX"
    else:
        sources = shapefile.Source.unique()
        if len(sources) == 0 or not sources[0]:
            #print(f"{filename} has no sources")
            if filename.startswith("MaxarImagery/HighFreq"):
                source = "MAX"
        elif len(sources) == 1:
            source = sources[0]
        elif len(sources) > 1:
            print(f"{filename} has ambiguous sources: {sources}")
            source = sources[0]
    n_lines = len(shapefile.dropna(subset="geometry").explode(index_parts=False))
    CPS = ""
    if "CPS" not in shapefile.columns:
        CPS = "No CPS column"
    else:
        CPS = shapefile.CPS.values
        if len(CPS) == 0:
            CPS = "No CPS values"
        elif 9 in CPS:
            CPS = "Has CPS=9"
        elif 0 in CPS:
            CPS = "Has CPS=0"
        elif None in CPS:
            CPS = "Has None CPS"
        elif pd.isna(CPS).any():
            CPS = "Has NA CPS"
        else:
            CPS = "OK"
    return {
        "Source": source,
        "n_lines": n_lines,
        "CPS": CPS,
        "Photoscale": "Photoscale" in shapefile.columns,
        "Pixel_Er": "Pixel_Er" in shapefile.columns
    }

shp_meta = pd.DataFrame(process_map(get_shapefile_meta, df.iterrows(), total=len(df)), index=df.index)
df = pd.concat([df, shp_meta], axis=1)
df

  0%|          | 0/2796 [00:00<?, ?it/s]

Retrolens/Waikato/OpitoBay/Shorelines/Opito_OtamaBay_14FEB2023.shp has ambiguous sources: ['PLE' 'MAX']
Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_06MAR2019.shp has ambiguous sources: ['LDS' None]
Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp has ambiguous sources: ['LDS' None]
Retrolens/Waikato/Whangamata/Shorelines/Whangamata_14FEB2023.shp has ambiguous sources: ['MAX' 'PLE']
Retrolens/Taranaki/OpunakeBeach/Shorelines/OpunakeBeach_11FEB2017.shp has ambiguous sources: ['LDS' None]
Retrolens/WestCoast/Westport/Shorelines/Westport_15JAN1988.shp has ambiguous sources: ['Rl' 'RL']
Retrolens/Northland/BayleysBeach/Shorelines/BayleysBeach_07APR2023.shp has ambiguous sources: ['MAX' None]
Retrolens/HawkesBay/Clifton_Haumoana/Shorelines/Clifton_Haumoana_23SEP1963.shp has ambiguous sources: ['RL' None]
MaxarImagery/HighFreq/Canterbury/PareoraRiver_Timaru/Shorelines/PareoraRiver_Timaru_11FEB2015.shp has ambiguous sources: ['MAX' None]MaxarImagery/HighFreq/Canterbury/PareoraR

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
33867,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
34689,Retrolens/Waikato/PortJackson/Shorelines/PortJackson_23AUG1971.shp,Retrolens/Northland/TaupiriBay/Stack/TaupiriBay_23AUG1971.tif,61.538462,RL,2,OK,False,False
38165,Retrolens/Waikato/Thames/Shorelines/Thames_25JAN1973.shp,Retrolens/Auckland/PakiriBeach_North/Stack/PakiriBeach_North_15JAN1977_mosaic.jp2,62.307692,RL,5,OK,False,False
33360,Retrolens/Waikato/Raglan/Shorelines/Raglan_24FEB2013.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_20FEB2016.tif,62.500000,WRC,2,OK,False,False
33343,Retrolens/Waikato/Raglan/Shorelines/Raglan_22MAR1944.shp,MaxarImagery/HighFreq/Otago/Aramoana/Imagery/Stack/Aramoana_20MAR2014.tif,62.500000,WRC,4,OK,False,False
...,...,...,...,...,...,...,...,...
48888,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.000000,RL,13,OK,True,True
48892,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_beach.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.000000,RL,8,OK,True,True
49079,Retrolens/Taranaki/UrenuiRiver_North_AOI/Shorelines/UrenuiRiver_North_5MAR1943.shp,Retrolens/Taranaki/UrenuiRiver_North_AOI/Stack/UrenuiRiver_North_5MAR1943_mosaic.jp2,100.000000,RLN,42,OK,True,True
48849,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_8NOV1970_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_8NOV1970_mosaic.jp2,100.000000,RL,9,OK,True,True


In [11]:
df.CPS.value_counts()

CPS
OK               2720
No CPS column      50
Has CPS=0          14
No CPS values      12
Name: count, dtype: int64

In [12]:
df[(df.CPS!="OK") & (df.n_lines > 0)][["filename", "Source", "n_lines", "CPS"]].sort_values(by=["CPS", "filename"]).to_csv(prefix+"Nick/CPS_issues.csv", index=False)

In [13]:
df.sort_values(by="match_score")

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
33867,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
34689,Retrolens/Waikato/PortJackson/Shorelines/PortJackson_23AUG1971.shp,Retrolens/Northland/TaupiriBay/Stack/TaupiriBay_23AUG1971.tif,61.538462,RL,2,OK,False,False
38165,Retrolens/Waikato/Thames/Shorelines/Thames_25JAN1973.shp,Retrolens/Auckland/PakiriBeach_North/Stack/PakiriBeach_North_15JAN1977_mosaic.jp2,62.307692,RL,5,OK,False,False
33360,Retrolens/Waikato/Raglan/Shorelines/Raglan_24FEB2013.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_20FEB2016.tif,62.500000,WRC,2,OK,False,False
33343,Retrolens/Waikato/Raglan/Shorelines/Raglan_22MAR1944.shp,MaxarImagery/HighFreq/Otago/Aramoana/Imagery/Stack/Aramoana_20MAR2014.tif,62.500000,WRC,4,OK,False,False
...,...,...,...,...,...,...,...,...
88966,Retrolens/Northland/BayofIslands_East/Shorelines/Bayofislands_East_22NOV1976.shp,Retrolens/Northland/BayofIslands_East/Stack/BayofIslands_East_22NOV1976.tif,100.000000,RL,36,OK,True,True
88979,Retrolens/Northland/BayofIslands_East/Shorelines/Bayofislands_East_22AUG1971.shp,Retrolens/Northland/BayofIslands_East/Stack/BayofIslands_East_22AUG1971.tif,100.000000,RL,12,OK,True,True
88984,Retrolens/Northland/BayofIslands_East/Shorelines/Bayofislands_East_17MAR1961.shp,Retrolens/Northland/BayofIslands_East/Stack/BayofIslands_East_17MAR1961.tif,100.000000,RL,27,OK,True,True
89783,Retrolens/Northland/CapeReinga/Shorelines/CapeReinga_08DEC2023.shp,MaxarImagery/HighFreq/Northland/CapeReinga/Stack/CapeReinga_08DEC2023.tif,100.000000,LDS,6,OK,False,False


In [14]:
failures = (df.match_score < 100) & (df.Source != "LDS") & (df.n_lines > 0)
print(sum(failures))
try:
  df[failures].to_csv(prefix+"Nick/failures.csv", index=False)
except:
  print("Can't write Nick/failures.csv")
df[failures]

407


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
33867,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
34689,Retrolens/Waikato/PortJackson/Shorelines/PortJackson_23AUG1971.shp,Retrolens/Northland/TaupiriBay/Stack/TaupiriBay_23AUG1971.tif,61.538462,RL,2,OK,False,False
38165,Retrolens/Waikato/Thames/Shorelines/Thames_25JAN1973.shp,Retrolens/Auckland/PakiriBeach_North/Stack/PakiriBeach_North_15JAN1977_mosaic.jp2,62.307692,RL,5,OK,False,False
33360,Retrolens/Waikato/Raglan/Shorelines/Raglan_24FEB2013.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_20FEB2016.tif,62.500000,WRC,2,OK,False,False
33343,Retrolens/Waikato/Raglan/Shorelines/Raglan_22MAR1944.shp,MaxarImagery/HighFreq/Otago/Aramoana/Imagery/Stack/Aramoana_20MAR2014.tif,62.500000,WRC,4,OK,False,False
...,...,...,...,...,...,...,...,...
97645,Retrolens/WestCoast/Hokitika_South/Shorelines/Hokitika_South_28MAR2023.shp,MaxarImagery/HighFreq/WestCoast/HokitikaNorth/Stack/HokitikaNorth_28MAR2023.tif,90.909091,MAX,3,OK,True,True
34527,Retrolens/Waikato/PortWaikato/Shorelines/PortWaikato_29MAR2012.shp,MaxarImagery/HighFreq/Waikato/PortWaikato/Imagery/Stack/PortWaikato_09MAR2016.tif,92.307692,RL,1,OK,False,False
33740,Retrolens/Waikato/Mokau/Shorelines/Mokau_12JAN2013.shp,MaxarImagery/HighFreq/Waikato/Mokau/Imagery/Stack/Mokau_11JAN2013.tif,92.857143,RL,1,OK,False,False
89903,Retrolens/Northland/HelenaBay/Shorelines/HelenaBay_12DEC1985.shp,Retrolens/Northland/HelenaBay/Stack/HelenaBay_13DEC1985.tif,94.444444,RL,48,OK,True,False


In [15]:
df.to_csv(prefix+"Nick/shoreline_image_matching.csv", index=False)

### For those perfect matches, get the metadata from the corresponding matched image

In [16]:
# Investigate metadata about the matched images

def get_meta(tup):
    i, row = tup
    try:
        image = rio.open(prefix + row.matched_image)
    except:
        print(f"Can't read {row.matched_image}")
        return {}
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.dropna(subset="geometry").explode(index_parts=False))
    except: 
        print(f"Can't read {row['filename']}")
        return {}
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if use_cache and os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, df[df.match_score == 100].iterrows(), total=sum(df.match_score == 100)))
    meta.to_csv(metafile, index=False)
meta

  0%|          | 0/1913 [00:00<?, ?it/s]

/home/nyou045/.local/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,compress,interleave,GCPs,res,photometric
0,Retrolens/Northland/Houhora/Shorelines/Houhora_20FEB1984.shp,Retrolens/Northland/Houhora/Stack/Houhora_20FEB1984.tif,100.0,RL,7,True,True,True,GTiff,uint16,65535.0,30456,32719,1,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.40232855969999476, 0.0, 1613948.1756023474, 0.0, -0.40232855969999476, 6149385.929406938, 0.0, 0.0, 1.0)",128.0,128,True,lzw,band,0,"(0.40232855969999476, 0.40232855969999476)",NaN
1,Retrolens/Northland/MarsdenPoint/Shorelines/MarsdenPoint_11MAY1968.shp,Retrolens/Northland/MarsdenPoint/Stack/MarsdenPoint_11MAY1968_mosiac.jp2,100.0,RL,1,True,True,True,JP2OpenJPEG,uint16,256.0,13769,13364,1,None,"(0.32666823639999026, 0.0, 1731198.1392688118, 0.0, -0.3266682363999882, 6033901.289773643, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,NaN,6,"(0.32666823639999026, 0.3266682363999882)",NaN
2,Retrolens/Northland/MarsdenPoint/Shorelines/MarsdenPoint_10JAN1979.shp,Retrolens/Northland/MarsdenPoint/Stack/MarsdenPoint_10JAN1979_mosaic.jp2,100.0,RL,1,True,True,True,JP2OpenJPEG,uint16,256.0,22123,28945,1,None,"(0.3432434214000076, 0.0, 1728272.2827226538, 0.0, -0.3432434214000093, 6035660.2212428935, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,NaN,6,"(0.3432434214000076, 0.3432434214000093)",NaN
3,Retrolens/Canterbury/OpihiRiver_RangitataRiver/Shorelines/OpihiRiver_RangitataRiver_9SEP1954.shp,Retrolens/Canterbury/OpihiRiver_RangitataRiver/Stack/OpihiRiver_RangitataRiver_9SEP1954_mosaic.jp2,100.0,RL,7,True,True,True,JP2OpenJPEG,uint16,256.0,17555,13288,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.5, 0.0, 1473975.5650565824, 0.0, -0.5, 5107925.721931805, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(0.5, 0.5)",NaN
4,Retrolens/Canterbury/OpihiRiver_OrariRiver/Shorelines/OpihiRiver_OrariRiver_19FEB1977.shp,Retrolens/Canterbury/OpihiRiver_OrariRiver/Stack/OpihiRiver_OrariRiver_19FEB1977_mosaic.jp2,100.0,RL,7,True,True,True,JP2OpenJPEG,uint16,256.0,10888,8385,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.7000000000000085, 0.0, 1467717.2863591702, 0.0, -0.7, 5101742.9872326655, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(0.7000000000000085, 0.7)",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.0,RL,13,True,True,True,JP2OpenJPEG,uint16,256.0,56494,32665,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.20000000000000082, 0.0, 1685313.7118251163, 0.0, -0.2, 5678879.025673549, 0.0, 0.0, 1.0)",NaN,32665,False,NaN,pixel,0,"(0.20000000000000082, 0.2)",NaN
1909,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_beach.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.0,RL,8,True,True,True,JP2OpenJPEG,uint16,256.0,56494,32665,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.20000000000000082, 0.0, 1685313.7118251163, 0.0, -0.2, 5678879.025673549, 0.0, 0.0, 1.0)",NaN,32665,False,NaN,pixel,0,"(0.20000000000000082, 0.2)",NaN
1910,Retrolens/Taranaki/UrenuiRiver_North_AOI/Shorelines/UrenuiRiver_North_5MAR1943.shp,Retrolens/Taranaki/UrenuiRiver_North_AOI/Stack/UrenuiRiver_North_5MAR1943_mosaic.jp2,100.0,RLN,42,True,True,True,JP2OpenJPEG,uint16,256.0,16369,17612,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.5, 0.0, 1719889.316493128, 0.0, -0.5, 5691295.472377276, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(0.5, 0.5)",NaN
1911,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_8NOV1970_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_8NOV1970_mosaic.jp2,100.0,RL,9,True,True,True,JP2OpenJPEG,uint16,256.0,20148,12973,3,"(proj, lat_0, lon_0, k, x_

## Read LDS index tiles

In [17]:
index_tiles = filelist[filelist.str.contains(".+index-tiles.+.shp$")]
index_tiles

18619    MaxarImagery/HighFreq/Auckland/Whatipu/Imagery/04JAN2022/auckland-0075m-rural-aerial-photos-index-tiles-2022/auckland-0075m-rural-aerial-photos-index-tiles-2022.shp
32226                                                                               Retrolens/Waikato/WaikawauRiver/waikato-05m-rural-aerial-photos-index-tiles-2012-2013.shp
33771                                                                                             Retrolens/Waikato/waikato-05m-rural-aerial-photos-index-tiles-2012-2013.shp
40679                                                                           Retrolens/Southland/ColacBay/LDS/southland-075m-rural-aerial-photos-index-tiles-2005-2011.shp
41450                                                                   Retrolens/Southland/Tiwai_Point/LDS/2007/southland-075m-rural-aerial-photos-index-tiles-2005-2011.shp
                                                                                         ...                                      

In [18]:
def read_index_tile(f):
    gdf = gpd.read_file(prefix+f).to_crs(2193)
    gdf["filename"] = f
    return gdf

index_tiles = pd.concat(process_map(read_index_tile, index_tiles))
len(index_tiles)

  0%|          | 0/64 [00:00<?, ?it/s]

131395

In [19]:
index_tiles.Captured = index_tiles.Captured.replace({"05, 06, 09, 10 Feb. 2014": "2014-02-05", "11 & 26 March 2014": "2014-03-11"})
index_tiles["parsed_date"] = pd.NA
for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:
    index_tiles["parsed_date"].update(pd.to_datetime(index_tiles[col].str.split(",|-|to").str[0], dayfirst=True, format="mixed", errors="ignore"))
index_tiles.parsed_date = index_tiles.parsed_date.astype("string")
index_tiles.parsed_date.value_counts(dropna=False)

/home/nyou045/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


parsed_date
2017                   23649
<NA>                   16486
2022                   11348
24/02/2017              2685
18/03/2020              2642
                       ...  
2016-11-15 15:12:28        1
2028-11-15 16:03:27        1
9/11/2016                  1
2029-03-12 13:01:12        1
2005-05-12 12:05:22        1
Name: count, Length: 363, dtype: Int64

## LDS index tile matching

In [20]:
LDS = df[(df.Source == "LDS") & (df.match_score < 100)].sort_values("match_score")
LDS

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
103423,Retrolens/Gisborne/Waiparapara/Shorelines/Waiparapara_07MAR2023.shp,MaxarImagery/HighFreq/Gisborne/WaipiroBay/Imagery/Stack/WaipiroBay_22MAR2003.tif,63.157895,LDS,6,OK,True,True
35746,Retrolens/Waikato/KaawaStream_North/Shorelines/KaawaStream_North_15FEB2022.shp,MaxarImagery/HighFreq/Tasman/TakakaRiver_North/Imagery/Stack/TakakaRiver_North_16FEB2003.tif,64.000000,LDS,5,OK,True,True
108080,Retrolens/Bay of Plenty/Houpoto/Shorelines/Houpoto_18JAN2022.shp,MaxarImagery/HighFreq/Taranaki/Oeo/Imagery/Stack/Oeo_18MAR2022.tif,64.285714,LDS,2,OK,True,True
101525,Retrolens/Gisborne/Tuparoa/Shorelines/Tuparoa_08FEB2023.shp,MaxarImagery/HighFreq/Tasman/Moturoa/Imagery/STACK/Moturoa_17FEB2018.tif,64.516129,LDS,11,OK,True,True
33841,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_04FEB2019.shp,MaxarImagery/HighFreq/Northland/Pataua/Imagery/Stack/Pataua_04FEB2010.tif,64.516129,LDS,4,OK,True,True
...,...,...,...,...,...,...,...,...
100587,Retrolens/Gisborne/TeAraroa/Shorelines/TeAraroa_19DEC2021.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_18DEC2021.tif,94.117647,LDS,11,OK,True,True
79957,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_01APR2006.shp,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Ryans_Pipikaretu_Penguin_TeRauoneBeach_17MAR2000_mosaic.jp2,94.252874,LDS,8,OK,True,True
41542,Retrolens/Southland/Tiwai_Point/Shorelines/TiwaiPoint_07FEB2016.shp,MaxarImagery/HighFreq/Southland/TiwaiPoint/Stack/TiwaiPoint_02FEB2016.tif,94.444444,LDS,1,OK,True,True
50103,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016_beach.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Stack/Oakura_03OCT2016.tif,96.551724,LDS,3,OK,True,True


In [21]:
def get_resolution(filename):
  gdf = gpd.read_file(prefix+filename)
  if "LDS" not in gdf.Source.unique():
    return {"filename": filename}
  bounds = gdf.total_bounds
  intersecting_tiles = index_tiles[index_tiles.intersects(box(*bounds))]
  if filename.startswith("Gabrielle"):
    intersecting_tiles = intersecting_tiles[intersecting_tiles.filename.str.startswith("Gabrielle")]
  if len(intersecting_tiles) == 0:
    print(f"{filename} doesn't intersect any index tiles")
    return {"filename": filename}
  if "Date" not in gdf.columns:
    #print(f"{filename} missing Date column")
    date = gdf.DSASdate.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  elif "DSASDate" not in gdf.columns:
    #print(f"{filename} missing DSASDate column")
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  else:
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASDate.unique()[0]
  if not DSASdate:
    DSASdate = date

  date_options = []
  date_to_col = {}
  for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:#, "parsed_date"]:
    options_for_col = intersecting_tiles[col].dropna().unique().tolist()
    date_options.extend(options_for_col)
    for date_option in options_for_col:
      date_to_col[date_option] = col
  display(date, DSASdate, gdf, intersecting_tiles.dropna(axis=1, how="all"), date_options, date_to_col)
  if not date_options:
    if "hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .1
      }
    else:
      print(f"No date options in {intersecting_tiles.filename.unique()}")
      return {"filename": filename}
  else:
    match = False
    if DSASdate in date_options:
      match = DSASdate
      score = 100
    for option in date_options:
      if DSASdate in option or date in option:
        match = option
        score = 100
    if not match:
      match, score, index = rapidfuzz.process.extractOne(query=date, choices=date_options, processor=lambda s: s.replace("-", ""))
    col_for_match = date_to_col[match]
    tiles_from_this_date = intersecting_tiles[intersecting_tiles[col_for_match] == match]
    GSDM = []
    for col in ['GSDM', 'ORTHO_GSD', 'Ortho_GSD', 'GSDm', 'GSD', 'GSD_M', 'GSD_CM', 'gsdM']:
      GSDM.extend(tiles_from_this_date[col].dropna().astype(str).str.strip("m").unique())
    if len(GSDM) == 0:
      tilefile = tiles_from_this_date.filename.unique()[0]
      if "-04m" in tilefile:
        GSDM = .4
      elif "-0075m" in tilefile:
        GSDM = .075
    elif len(GSDM) == 1:
      GSDM = GSDM[0]
    elif len(GSDM) > 1:
      #print(f"Ambiguous GSDM: {GSDM}")
      GSDM = GSDM[0]
    return {
      "filename": filename,
      "matched_index_tiles": tiles_from_this_date.filename.unique().tolist(),
      "date": date,
      "DSASDate": DSASdate,
      "matched_date": match,
      "matched_date_col": col_for_match,
      "match_score": score,
      "Pixel_ER": GSDM
    }

#get_resolution("Retrolens/Taranaki/PariokariwaPoint/Shorelines/PariokariwaPoint_19MAR2017_Taranaki399_cliff.shp")
LDS = pd.DataFrame(process_map(get_resolution, LDS.filename)).sort_values("match_score")
print("Perfect matches:", sum(LDS.match_score == 100))
print("Imperfect matches:", sum(LDS.match_score < 100))
LDS

  0%|          | 0/476 [00:00<?, ?it/s]

No date options in ['SpatialData/Mosaics/Footprint shapefiles/BayofPlenty/bay-of-plenty-0125m-urban-aerial-photos-index-tiles-2014-201.shp']


'2019-02-11'

'11/02/2019'

,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,geometry
0,0,Waikato,KawaaStream_North,2019-02-11,11/02/2019,AF,1000,took the latest date for flown,LDS,2,1,None,0,0.3,0.78924,"LINESTRING (1750956.870 5855009.718, 1750969.111 5854997.789, 1750979.430 5854979.533, 1750987.896 5854970.801, 1750990.542 5854969.214, 1750999.538 5854968.420, 1751005.623 5854966.568, 1751008.0..."
1,0,Waikato,KawaaStream_North,2019-02-11,11/02/2019,AF,1000,took the latest date for flown,LDS,2,1,None,0,0.3,0.78924,"LINESTRING (1751526.391 5854558.183, 1751532.741 5854513.203, 1751535.122 5854480.130, 1751534.857 5854459.228, 1751550.997 5854421.393, 1751565.020 5854398.109, 1751565.549 5854371.387, 1751558.4..."
2,0,Waikato,KawaaStream_North,2019-02-11,11/02/2019,AF,1000,took the latest date for flown,LDS,2,1,None,0,0.3,0.78924,"LINESTRING (1752193.671 5852497.339, 1752227.009 5852500.779, 1752260.875 5852497.604, 1752277.280 5852494.693, 1752296.594 5852478.289, 1752309.559 5852458.445, 1752318.025 5852441.512, 1752324.9..."
3,0,Waikato,KawaaStream_North,2019-02-11,11/02/2019,AF,1000,took the latest date for flown,LDS,2,1,None,0,0.3,0.78924,"LINESTRING (1752420.155 5852310.278, 1752400.576 5852305.251, 1752383.907 5852298.107, 1752376.234 5852294.932, 1752370.942 5852290.964, 1752363.005 5852287.259, 1752351.892 5852282.761, 1752343.1..."
4,0,Waikato,KawaaStream_North,2019-02-11,11/02/2019,AF,1000,took the latest date for flown,LDS,2,1,None,0,0.3,0.78924,"LINESTRING (1752524.665 5851255.118, 1752552.711 5851258.557, 1752572.026 5851269.405, 1752584.726 5851269.405, 1752603.511 5851266.759, 1752614.359 5851265.701, 1752631.557 5851263.320, 1752658.8..."


...

,TILENAME,MAPSHEET,SCALE,TILE,YEAR,FLOWN,geometry,filename,SHEET,PROJECT,SUPPLIER,ACC_XY,GSDm,HCOORDSYS,CAMERA,VDATUM,GSDM,SURVEY,ACCCURACY,parsed_date
736,BH29_5000_0701,BH29,5000,0701,2022,"14/01/2022, 16/01/2022","POLYGON ((1686400.000 5668800.000, 1684000.000 5668800.000, 1684000.000 5672400.000, 1686400.000 5672400.000, 1686400.000 5668800.000))",SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-025m-rural-aerial-photos-index-tiles-2021-2022.shp,NaN,TRAPP 2021/22 Aerial Photograph,AERIAL SURVEYS LTD,NaN,NaN,NZTM,UCE-f100,NZVD2016,0.25,SN14207,+/-0.4 at 95% confidence level in clear flat open areas (2 sigma),14/01/2022
737,BH29_5000_0601,BH29,5000,0601,2022,"16/01/2022, 14/01/2022","POLYGON ((1686400.000 5672400.000, 1684000.000 5672400.000, 1684000.000 5676000.000, 1686400.000 5676000.000, 1686400.000 5672400.000))",SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-025m-rural-aerial-photos-index-tiles-2021-2022.shp,NaN,TRAPP 2021/22 Aerial Photograph,AERIAL SURVEYS LTD,NaN,NaN,NZTM,UCE-f100,NZVD2016,0.25,SN14207,+/-0.4 at 95% confidence level in clear flat open areas (2 sigma),16/01/2022
822,BH28_5000_0710,BH28,5000,0710,2022,14/01/2022,"POLYGON ((1684000.000 5668800.000, 1681600.000 5668800.000, 1681600.000 5672400.000, 1684000.000 5672400.000, 1684000.000 5668800.000))",SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-025m-rural-aerial-photos-index-tiles-2021-2022.shp,NaN,TRAPP 2021/22 Aerial Photograph,AERIAL SURVEYS LTD,NaN,NaN,NZTM,UCE-f100,NZVD2016,0.25,SN14207,+/-0.4 at 95% confidence level in clear flat open areas (2 sigma),14/01/2022
875,BH28_5000_0610,BH28,5000,0610,2022,14/01/2022,"POLYGON ((1681600.000 5676000.000, 1684000.000 5676000.000, 1684000.000 5672400.000, 1681600.000 5672400.000, 1681600.000 5676000.000))",SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-025m-rural-aerial-photos-index-tiles-2021-2022.shp,NaN,TRAPP 2021/22 Aerial Photograph,AERIAL SURVEYS LTD,NaN,NaN,NZTM,UCE-f100,NZVD2016,0.25,SN14207,+/-0.4 at 95% confidence level in clear flat open areas (2 sigma),14/01/2022
300,BH28_2K_1625,NaN,2k,1625,2016,31/10/2016,"POLYGON ((1683040.000 5672400.000, 1684000.000 5672400.000, 1684000.000 5670960.000, 1683040.000 5670960.000, 1683040.000 5672400.000))",SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp,BH28,TARANAKI REGIONAL AERIAL PHOTOGRAPHY PROJECT (TRAPP) 2016-17,AAM NZ Limited (AAM),+/-2.0m @ 90% confidence level,0.3m,NZTM GD2000,NaN,Taranaki 1970,NaN,NaN,NaN,31/10/2016
301,BH28_2K_1725,NaN,2k,1725,2016,31/10/2016,"POLYGON ((1683040.000 5670960.000, 1684000.000 5670960.000, 1684000.000 5669520.000, 1683040.000 5669520.000, 1683040.000 5670960.000))",SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp,BH28,TARANAKI REGIONAL AERIAL PHOTOGRAPHY PROJECT (TRAPP) 2016-17,AAM NZ Limited (AAM),+/-2.0m @ 90% confidence level,0.3m,NZTM GD2000,NaN,Taranaki 1970,NaN,NaN,NaN,31/10/2016
310,BH28_2K_1724,NaN,2k,1724,2016,31/10/2016,"POLYGON ((1683040.000 5669520.000, 1682080.000 5669520.000, 1682080.000 5670960.000, 1683040.000 5670960.000, 1683040.000 5669520.000))",SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp,BH28,TARANAKI REGIONAL AERIAL PHOTOGRAPHY PROJECT (TRAPP) 2016-17,AAM NZ Limited (AAM),+/-2.0m @ 90% confidence level,0.3m,NZTM GD2000,NaN,Taranaki 1970,NaN,NaN,NaN,31/10/2016
776,BH29_2K_1502,NaN,2k,1502,2016,31/10/2016,"POLYGON ((1685920.000 5672400.000, 1684960.000 5672400.000, 1684960.000 5673840.000, 1685920.000 5673840.000, 1685920.000 5672400.000))",SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp,BH29,TARANAKI REGIONAL AERIAL PHOTOGRAPHY PROJECT (TRAPP) 2016-17,AAM NZ Limited (AAM),+/-2.0m @ 90% confidence level,0.3m,NZTM GD2000,NaN,T

['14/01/2022, 16/01/2022',
 '16/01/2022, 14/01/2022',
 '14/01/2022',
 '31/10/2016']

{'14/01/2022, 16/01/2022': 'FLOWN',
 '16/01/2022, 14/01/2022': 'FLOWN',
 '14/01/2022': 'FLOWN',
 '31/10/2016': 'FLOWN'}

Perfect matches: 265
Imperfect matches: 204


,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
54,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
131,Retrolens/Gisborne/Gisborne/Shorelines/Gisborne_07MAR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-gisborne-03m-rural-aerial-photos-index-tiles-2017-2019-SHP (1)/gisborne-03m-rural-aerial-photos-index-tiles-2017-2019.shp],2023-03-07,07/03/2023,16/02/2018,FLOWN,33.333333,0.3
235,Retrolens/Bay of Plenty/OmaioBay/Shorelines/OmaioBay_20DEC2021.shp,[SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp],2021-12-20,20/12/2021,03/05/2019,FLOWN,33.333333,0.3
114,Retrolens/Bay of Plenty/PapateaBay/Shorelines/PapateaBay_20DEC2021.shp,[SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp],2021-12-20,20/12/2021,03/05/2019,FLOWN,33.333333,0.3
222,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
...,...,...,...,...,...,...,...,...
93,Retrolens/Bay of Plenty/Matata/Shorelines/Matata_31MAR2023.shp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,Retrolens/Bay of Plenty/Matata/Shorelines/Matata_13DEC2014.shp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,Retrolens/Bay of Plenty/HikuwaiBeach/Shorelines/HikuwaiBeach_20DEC2021.shp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,Retrolens/Bay of Plenty/KohioawaBeach/Shorelines/KohioawaBeach_03DEC2014.shp,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
with pd.option_context("display.max_rows", None):
  display(LDS[LDS.match_score < 100])

,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
54,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
131,Retrolens/Gisborne/Gisborne/Shorelines/Gisborne_07MAR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-gisborne-03m-rural-aerial-photos-index-tiles-2017-2019-SHP (1)/gisborne-03m-rural-aerial-photos-index-tiles-2017-2019.shp],2023-03-07,07/03/2023,16/02/2018,FLOWN,33.333333,0.3
235,Retrolens/Bay of Plenty/OmaioBay/Shorelines/OmaioBay_20DEC2021.shp,[SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp],2021-12-20,20/12/2021,03/05/2019,FLOWN,33.333333,0.3
114,Retrolens/Bay of Plenty/PapateaBay/Shorelines/PapateaBay_20DEC2021.shp,[SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp],2021-12-20,20/12/2021,03/05/2019,FLOWN,33.333333,0.3
222,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
272,Retrolens/Bay of Plenty/PukehinaBeach/Shorelines/PukehinaBeach_04APR2023.shp,[SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2018-2019.shp],2023-04-04,04/04/2023,07/12/2018_17/01/2019,FLOWN,36.000000,0.1
301,Retrolens/Bay of Plenty/MountMaunganui/Shorelines/MountMaunganui_23MAR2023.shp,[SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2018-2019.shp],2023-03-23,23/03/2023,07/12/2018_17/01/2019,FLOWN,36.000000,0.1
233,Retrolens/Bay of Plenty/MountMaunganui/Shorelines/MountMaunganui_04APR2023.shp,[SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2018-2019.shp],2023-04-04,04/04/2023,07/12/2018_17/01/2019,FLOWN,36.000000,0.1
159,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
298,Retrolens/Canterbury/OpihiRiver_RangitataRiver/Shorelines/OpihiRiver_RangitataRiver_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3


In [23]:
# Stop Excel interpreting dates
for col in ["date", "DSASDate", "matched_date"]:
    LDS[col] = "'" + LDS[col]
LDS.to_csv(prefix + "Nick/LDS_matches.csv", index=False)